In [1]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('../')

In [3]:
import pyaudio
import wave
import threading
import time
import os
import datetime

In [4]:
class Audio():
    
    def __init__(self,stream_name,bitrate,frames_per_buffer,channels):
        self.name=stream_name
        self.bitrate=bitrate
        self.frames_per_buffer=frames_per_buffer
        self.channels=channels
        self.audio_format=pyaudio.paInt16
        self.is_running=False
        self.is_recording=False
        
    def start(self):
        if self.is_running:
            return
        self.is_running=True
        self.audio_filename = f"{self.name}.wav"
        self.audio = pyaudio.PyAudio()
        self.stream = self.audio.open(format=self.audio_format,
                                      channels=self.channels,
                                      rate=self.bitrate,
                                      input=True,
                                      frames_per_buffer = self.frames_per_buffer)
        self.stream.start_stream()
        self.audio_frames = []

    def start_recording(self,output_path):
        if self.is_recording:
            return
        self.output_path=output_path
        if not os.path.isdir(output_path):
            os.mkdir(output_path)
        self.output_path=output_path
        self.record_start_time = time.time()
        self.is_recording=True
        self.record_thread = threading.Thread(target=self.run_record, args=())
        self.record_thread.daemon = True
        self.record_thread.start()      

    def run_record(self):
        while self.is_recording:
            try:
                data = self.stream.read(self.frames_per_buffer) 
                self.audio_frames.append(data)
            except:
                pass       
            
    def stop_recording(self):
        if not self.is_recording:
            return
        self.is_recording=False
        self.record_end_time=time.time()
        self.record_thread.join()  
        print('Audio record thread ended')
        
        print('Saving audio file')
        waveFile = wave.open(self.output_path+'/'+self.audio_filename, 'wb')
        waveFile.setnchannels(self.channels)
        waveFile.setsampwidth(self.audio.get_sample_size(self.audio_format))
        waveFile.setframerate(self.bitrate)
        waveFile.writeframes(b''.join(self.audio_frames))
        waveFile.close()
        print('closed audio file')
        
    # Finishes the audio recording therefore the thread too    
    def stop(self):
        self.stop_recording()
        self.is_running=False
        
        self.stream.stop_stream()
        self.stream.close()
        self.audio.terminate()
        

In [10]:
audio=Audio('Microphone',bitrate=44100,frames_per_buffer=1024,channels=2)

In [11]:
audio.start()

In [12]:
audio.start_recording('output')

In [13]:
audio.stop_recording()

Audio record thread ended
Saving audio file
closed audio file


In [9]:
audio.stop()